### Libraries

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import opti_recruit.feature_engineering as fe
import opti_recruit.get_team_features as gtf
import opti_recruit.similarity as si
import pandas as pd
import pickle 
from sklearn.pipeline import Pipeline,make_pipeline,make_union
from sklearn.compose import make_column_transformer,make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from opti_recruit.data import get_data, clean_data
import numpy as np

### Import data

#### Fifa 22

In [3]:

df22 = get_data()[22]
df22.head()

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,78000000.0,320000.0,34,...,50+3,50+3,50+3,61+3,19+3,https://cdn.sofifa.net/players/158/023/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png
1,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,92,92,119500000.0,270000.0,32,...,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/22_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1353/60.png,https://cdn.sofifa.net/flags/pl.png
2,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",91,91,45000000.0,270000.0,36,...,53+3,53+3,53+3,60+3,20+3,https://cdn.sofifa.net/players/020/801/22_120.png,https://cdn.sofifa.net/teams/11/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png
3,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,129000000.0,270000.0,29,...,50+3,50+3,50+3,62+3,20+3,https://cdn.sofifa.net/players/190/871/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,NaN,https://cdn.sofifa.net/flags/br.png
4,192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91,125500000.0,350000.0,30,...,69+3,69+3,69+3,75+3,21+3,https://cdn.sofifa.net/players/192/985/22_120.png,https://cdn.sofifa.net/teams/10/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png


#### FBref player list 

In [4]:
json_player = pd.read_json('../raw_data/sofifa_ids.json').rename(columns={0: "sofifa_id"})
json_player.shape

(7717, 1)

In [5]:
list_json_player = list(json_player.sofifa_id)
list_json_player

[193041,
 158023,
 190871,
 230621,
 20801,
 188545,
 189596,
 167495,
 212622,
 209658,
 239085,
 202652,
 192985,
 209331,
 208722,
 176580,
 192505,
 215914,
 155862,
 210257,
 195864,
 167948,
 202126,
 192119,
 182521,
 177003,
 192448,
 153079,
 200389,
 207865,
 186153,
 211110,
 192387,
 212198,
 212831,
 216267,
 200145,
 188350,
 178603,
 192984,
 177683,
 218667,
 210514,
 212218,
 193747,
 188377,
 186942,
 204485,
 231866,
 233049,
 201535,
 179813,
 188567,
 163587,
 208830,
 183277,
 189511,
 228702,
 226161,
 194765,
 168542,
 208093,
 198219,
 201024,
 216352,
 232363,
 138956,
 209499,
 239818,
 184344,
 189513,
 189509,
 164240,
 181291,
 206113,
 204923,
 209889,
 208574,
 207862,
 206517,
 183711,
 201942,
 208670,
 212188,
 205988,
 231677,
 226790,
 207863,
 200458,
 197445,
 146536,
 202556,
 41236,
 204639,
 186345,
 175943,
 215698,
 205498,
 231478,
 235243,
 200647,
 198706,
 193080,
 184134,
 199434,
 202811,
 192629,
 152729,
 230142,
 210243,
 200610,
 18

#### Filter df22 with json_player



In [6]:
boolean_series = df22.sofifa_id.isin(list_json_player)
filtered_df22 = df22[boolean_series]
filtered_df22.shape

(6760, 110)

#### Add engineered feature for similarity matrix


In [7]:
df22_boost = fe.add_features(filtered_df22)


### Data Preparation

#### Prepare Numeric Dataframe

In [8]:
to_drop = ['sofifa_id','short_name','player_positions','height_cm','weight_kg','club_team_id'
          ,'club_name' ,'league_name','club_position','club_joined','club_contract_valid_until','nationality_name'
          ,'nation_team_id','preferred_foot','weak_foot','work_rate','body_type','player_tags','player_traits'
          ,'is_bench','potential_diff','age_bin','player_pos','new_nationality'
          ,'value_eur','wage_eur','release_clause_eur']
#             'age']

In [9]:
df22_num = df22_boost.drop(to_drop, axis = 1)

#### Pipeline

In [10]:
from sklearn.pipeline import Pipeline,make_pipeline,make_union
from sklearn.compose import make_column_transformer,make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn import set_config; set_config(display='diagram')



num_transformer = make_pipeline(SimpleImputer(), StandardScaler())
num_col = make_column_selector(dtype_include=['float64','int64'])

cat_transformer = OneHotEncoder()
cat_col = make_column_selector(dtype_include=['object','category'])

preproc_basic = make_column_transformer(
    (num_transformer, num_col),
    (cat_transformer, cat_col),
    remainder='passthrough')

In [11]:
SimpleImputer.get_feature_names_out = (lambda self, names=None: self.feature_names_in_)

#### Pipeline DF

In [12]:
X_train_transformed = preproc_basic.fit_transform(df22_num)


test_df = pd.DataFrame(X_train_transformed, 
             columns=preproc_basic.get_feature_names_out()
            )


### Cosine Similarity

#### Raw sim matrix

In [13]:
from sklearn.metrics.pairwise import cosine_similarity 
similarities = cosine_similarity(test_df)

#### Misc functions

In [14]:
def normalize(array):
    return np.array([round(num, 2) for num in (array - min(array))*100/(max(array)-min(array))])

def get_index(x):
    return filtered_df22[filtered_df22['sofifa_id']==x].index.tolist()[0]

#### Testing

In [15]:
index = get_index(158023)
index

0

In [39]:
## list of reco for a specfic player -- (similarities[index,:]) 
## list of reco for a specfic player, sorted by index DESC -- similarities[index,:].argsort()
## list of reco for a specfic player, sorted by index ASC -- similarities[index,:].argsort()[::-1]
## list of reco for specfic player, sorted by index ASC, without player itself -- similarities[index,:].argsort()[::-1]

In [17]:
d = {
    'index' : index
    ,'scores':np.sort(normalize(similarities[index,:]))[-101:][::-1][1:]
    ,'reco_player_index': similarities[index,:].argsort()[-101:][::-1][1:]
}

d

{'index': 0,
 'scores': array([97.88, 97.48, 97.36, 97.1 , 96.87, 96.82, 96.81, 96.7 , 96.68,
        96.67, 96.61, 96.59, 96.58, 96.5 , 96.49, 96.37, 96.05, 95.97,
        95.8 , 95.62, 95.61, 95.58, 95.52, 95.46, 95.45, 95.43, 95.41,
        95.29, 95.26, 95.15, 95.15, 95.14, 95.12, 95.1 , 95.08, 95.04,
        94.97, 94.71, 94.7 , 94.67, 94.63, 94.62, 94.53, 94.42, 94.36,
        94.29, 94.26, 94.24, 94.17, 94.09, 94.07, 94.05, 94.  , 93.98,
        93.89, 93.82, 93.8 , 93.72, 93.65, 93.64, 93.57, 93.56, 93.41,
        93.4 , 93.39, 93.37, 93.37, 93.36, 93.33, 93.32, 93.25, 93.19,
        93.18, 93.16, 93.12, 93.06, 92.99, 92.88, 92.87, 92.86, 92.85,
        92.81, 92.78, 92.74, 92.73, 92.67, 92.63, 92.62, 92.59, 92.56,
        92.55, 92.52, 92.51, 92.44, 92.44, 92.4 , 92.32, 92.31, 92.23,
        92.22]),
 'reco_player_index': array([  73,    3,   16,   77,   32,  376,   40,   11,  175,   63,    2,
          57,   29,   86,   81,  129,   26,   79,   15,    6,  321,  395,
          

In [18]:
len(d['scores'])

100

#### Similarity : List of dict for each player ( index looked up player , scores, index of reco players)

In [19]:
from tqdm import tqdm_notebook as tqdm
import ipywidgets 

In [ ]:
new_simili = []
for i in tqdm(range(0,len(similarities))):
# for i in range(0,2):
    d = {
        'index':i
        ,'scores':np.sort(normalize(similarities[i,:]))[-101:][::-1][1:]
        ,'reco_player_index': similarities[i,:].argsort()[-101:][::-1][1:]
    }
    new_simili.append(d)


/var/folders/tc/0tkkgmdd25n0rh3_pjfhw6mw0000gp/T/ipykernel_30145/3647775946.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(0,len(similarities))):


  0%|          | 0/6760 [00:00<?, ?it/s]

In [47]:
(new_simili[0])

{'index': 0,
 'scores': array([9.788e+01, 9.748e+01, 9.736e+01, ..., 1.100e-01, 7.000e-02,
        0.000e+00]),
 'reco_player_index': array([  73,    3,   16, ..., 6713, 6614, 6509])}

#### Dataframe of the list of dict 

In [48]:
sim_df_test = pd.DataFrame.from_dict(new_simili)

In [49]:
sim_df_test

,index,scores,reco_player_index
0,0,"[97.88, 97.48, 97.36, 97.1, 96.87, 96.82, 96.8...","[73, 3, 16, 77, 32, 376, 40, 11, 175, 63, 2, 5..."
1,1,"[98.79, 98.12, 98.05, 97.72, 97.67, 97.52, 97....","[22, 2, 11, 29, 35, 9, 207, 15, 72, 24, 81, 25..."
2,2,"[98.14, 97.56, 97.44, 97.34, 97.09, 96.92, 96....","[1, 29, 11, 207, 15, 79, 81, 6, 0, 498, 22, 35..."
3,3,"[98.67, 97.82, 97.82, 97.64, 97.44, 97.42, 97....","[73, 26, 32, 16, 0, 86, 118, 63, 77, 15, 175, ..."
4,4,"[98.53, 98.42, 97.79, 97.31, 97.23, 96.99, 96....","[113, 27, 76, 288, 36, 31, 166, 119, 98, 80, 3..."
...,...,...,...
6755,6755,"[96.83, 95.74, 94.78, 93.93, 93.46, 93.23, 93....","[6749, 6748, 6724, 6602, 6745, 6754, 6645, 674..."
6756,6756,"[98.5, 97.54, 97.37, 96.85, 96.7, 96.44, 96.07...","[6706, 6523, 6671, 6716, 6453, 6688, 6559, 672..."
6757,6757,"[97.87, 97.05, 96.89, 96.81, 96.05, 95.91, 94....","[6747, 6697, 6724, 6739, 6749, 6645, 6649, 670..."
6758,6758,"[94.1, 92.38, 92.02, 91.83, 91.83, 91.21, 90.9...","[6674, 6576, 6631, 6591, 6748, 6754, 6397, 654..."


In [50]:
with open(r'../similarity_matrix_v2.pickle', 'wb') as file:
        pickle.dump(sim_df_test, file)

### Method to construct list of reco dictionaries for a given player 

#### Load pickle DF

In [52]:
with open("../similarity_matrix_v2.pickle", 'rb') as file:
    sim_matrix_df = pickle.load(file)

In [56]:
sim_matrix_df.scores.head()

0    [97.88, 97.48, 97.36, 97.1, 96.87, 96.82, 96.8...
1    [98.79, 98.12, 98.05, 97.72, 97.67, 97.52, 97....
2    [98.14, 97.56, 97.44, 97.34, 97.09, 96.92, 96....
3    [98.67, 97.82, 97.82, 97.64, 97.44, 97.42, 97....
4    [98.53, 98.42, 97.79, 97.31, 97.23, 96.99, 96....
Name: scores, dtype: object

In [58]:
print(sim_matrix_df.scores[1])
type(sim_matrix_df.scores[1])
len(sim_matrix_df.reco_player_index[0])

[9.879e+01 9.812e+01 9.805e+01 ... 7.000e-02 4.000e-02 0.000e+00]


6759

#### Create the list for index 0 and transform into Dataframe


In [59]:
index_search = 0 

length = len(sim_matrix_df.reco_player_index[index_search])

list_res=[]
for i in range(0,length):
    d = {
        'index_search' : index_search,
        'index' : sim_matrix_df.reco_player_index[index_search][i],
        'score': sim_matrix_df.scores[index_search][i]
        }
    list_res.append(d)
list_res

[{'index_search': 0, 'index': 73, 'score': 97.88},
 {'index_search': 0, 'index': 3, 'score': 97.48},
 {'index_search': 0, 'index': 16, 'score': 97.36},
 {'index_search': 0, 'index': 77, 'score': 97.1},
 {'index_search': 0, 'index': 32, 'score': 96.87},
 {'index_search': 0, 'index': 376, 'score': 96.82},
 {'index_search': 0, 'index': 40, 'score': 96.81},
 {'index_search': 0, 'index': 11, 'score': 96.7},
 {'index_search': 0, 'index': 175, 'score': 96.68},
 {'index_search': 0, 'index': 63, 'score': 96.67},
 {'index_search': 0, 'index': 2, 'score': 96.61},
 {'index_search': 0, 'index': 57, 'score': 96.59},
 {'index_search': 0, 'index': 29, 'score': 96.58},
 {'index_search': 0, 'index': 86, 'score': 96.5},
 {'index_search': 0, 'index': 81, 'score': 96.49},
 {'index_search': 0, 'index': 129, 'score': 96.37},
 {'index_search': 0, 'index': 26, 'score': 96.05},
 {'index_search': 0, 'index': 79, 'score': 95.97},
 {'index_search': 0, 'index': 15, 'score': 95.8},
 {'index_search': 0, 'index': 6, '

In [68]:
my_reco = pd.DataFrame(list_res)

my_reco['sofifa_id']=df22.iloc[my_reco['index']]['sofifa_id']


In [73]:
sofifa_index = df22[['sofifa_id']]
sofifa_index

,sofifa_id
0,158023
1,188545
2,20801
3,190871
4,192985
...,...
19234,261962
19235,262040
19236,262760
19237,262820


In [69]:
my_reco.head(10)

,index_search,index,score,sofifa_id
0,0,73,97.88,NaN
1,0,3,97.48,188545.0
2,0,16,97.36,20801.0
3,0,77,97.10,190871.0
4,0,32,96.87,192985.0
5,0,376,96.82,200389.0
6,0,40,96.81,231747.0
7,0,11,96.70,167495.0
8,0,175,96.68,192448.0
9,0,63,96.67,202126.0


In [65]:
df22.head(20)

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,78000000.0,320000.0,34,...,50+3,50+3,50+3,61+3,19+3,https://cdn.sofifa.net/players/158/023/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png
1,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,92,92,119500000.0,270000.0,32,...,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/22_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1353/60.png,https://cdn.sofifa.net/flags/pl.png
2,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",91,91,45000000.0,270000.0,36,...,53+3,53+3,53+3,60+3,20+3,https://cdn.sofifa.net/players/020/801/22_120.png,https://cdn.sofifa.net/teams/11/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png
3,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,129000000.0,270000.0,29,...,50+3,50+3,50+3,62+3,20+3,https://cdn.sofifa.net/players/190/871/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,NaN,https://cdn.sofifa.net/flags/br.png
4,192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91,125500000.0,350000.0,30,...,69+3,69+3,69+3,75+3,21+3,https://cdn.sofifa.net/players/192/985/22_120.png,https://cdn.sofifa.net/teams/10/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png
5,200389,https://sofifa.com/player/200389/jan-oblak/220002,J. Oblak,Jan Oblak,GK,91,93,112000000.0,130000.0,28,...,33+3,33+3,33+3,32+3,89+3,https://cdn.sofifa.net/players/200/389/22_120.png,https://cdn.sofifa.net/teams/240/60.png,https://cdn.sofifa.net/flags/es.png,NaN,https://cdn.sofifa.net/flags/si.png
6,231747,https://sofifa.com/player/231747/kylian-mbappe...,K. Mbappé,Kylian Mbappé Lottin,"ST, LW",91,95,194000000.0,230000.0,22,...,54+3,54+3,54+3,63+3,18+3,https://cdn.sofifa.net/players/231/747/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,https://cdn.sofifa.net/teams/1335/60.png,https://cdn.sofifa.net/flags/fr.png
7,167495,https://sofifa.com/player/167495/manuel-neuer/...,M. Neuer,Manuel Peter Neuer,GK,90,90,13500000.0,86000.0,35,...,34+3,34+3,34+3,35+3,88+2,https://cdn.sofifa.net/players/167/495/22_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1337/60.png,https://cdn.sofifa.net/flags/de.png
8,192448,https://sofifa.com/player/192448/marc-andre-te...,M. ter Stegen,Marc-André ter Stegen,GK,90,92,99000000.0,250000.0,29,...,33+3,33+3,33+3,31+3,88+3,https://cdn.sofifa.net/players/192/448/22_120.png,https://cdn.sofifa.net/teams/241/60.png,https://cdn.sofifa.net/flags/es.png,NaN,https://cdn.sofifa.net/flags/de.png
9,202126,https://sofifa.com/player/202126/harry-kane/22...,H. Kane,Harry Kane,ST,90,90,129500000.0,240000.0,27,...,61+3,61+3,61+3,64+3,20+3,https://cdn.sofifa.net/players/202/126/22_120.png,https://cdn.sofifa.net/teams/18/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1318/60.png,https://cdn.sofifa.net/flags/gb-eng.png
